In [1]:
import requests, pandas as pd, numpy as np
import csv
import time
import urllib.parse
#API_KEY=
#gameName=
#tagLine=
#region=
def read_secrets(file_path):
    with open(file_path) as f:
        return {
            line.split('=', 1)[0].strip(): line.split('=', 1)[1].strip()
            for line in f
            if '=' in line and not line.strip().startswith('#') and line.strip()
        }

# Example usage
secret = read_secrets('.env')
# --- Your secrets ---
API_KEY = secret['API_KEY']
gameName = secret['gameName']
tagLine = secret['tagLine']
region = secret['region']

# --- Headers ---
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.8",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API_KEY
    }


secret


{'API_KEY': 'RGAPI-2f57bf54-0e1f-438e-af0d-72954d9c77d2',
 'gameName': 'areanyaaaa',
 'tagLine': 'pfp',
 'region': 'americas'}

In [2]:
def get_puuid_by_riot_id(region, gameName,tagLine, HEADERS):
    url = f"https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}"
    r = requests.get(url, headers=HEADERS)
    if r.status_code == 200:
        return r.json()
    else:
        return f"[Error] get_summoner_by_name: {r.status_code} {r.text}"

In [3]:
response = get_puuid_by_riot_id(region,gameName,tagLine,HEADERS)
puuid = response['puuid']
puuid

'dY1iOMlz4B5hASgR0lrkJbswM6aYq7viNgjAIkM6XJhAviNcCLX5xj9HRgImWCWCgLzmfW3fLCIVUA'

In [4]:
def get_account_by_puuid(region, puuid, HEADERS):
    url = f"https://{region}.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}"
    r = requests.get(url, headers=HEADERS)
    if r.status_code == 200:
        return r.json()
    else:
        return f"[Error] get_account_by_puuid: {r.status_code} {r.text}"


In [5]:
get_account_by_puuid(region,puuid,HEADERS)

{'puuid': 'dY1iOMlz4B5hASgR0lrkJbswM6aYq7viNgjAIkM6XJhAviNcCLX5xj9HRgImWCWCgLzmfW3fLCIVUA',
 'gameName': 'areanyaaaa',
 'tagLine': 'pfp'}

In [6]:
import requests

def get_tft_match_ids_by_puuid(region, puuid, HEADERS, count=20):
    url = f"https://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count={count}"
    r = requests.get(url, headers=HEADERS)
    if r.status_code == 200:
        return r.json()
    else:
        return f"[Error] get_tft_match_ids_by_puuid: {r.status_code} {r.text}"


In [10]:
listtOfMatches = get_tft_match_ids_by_puuid(region, puuid, HEADERS, count=20)
listtOfMatches

['PBE1_4508747974',
 'PBE1_4508741314',
 'PBE1_4508732536',
 'PBE1_4508725346',
 'PBE1_4508721695',
 'PBE1_4508417790',
 'PBE1_4508413529',
 'PBE1_4508405374',
 'PBE1_4508397917',
 'PBE1_4508389167',
 'PBE1_4508362809',
 'PBE1_4508351139',
 'PBE1_4508193603',
 'PBE1_4508185504',
 'PBE1_4508177374',
 'PBE1_4508169801',
 'PBE1_4508163919',
 'PBE1_4508159760',
 'PBE1_4508156095',
 'PBE1_4508149823']

In [8]:
def get_tft_match_by_id(region, matchId, HEADERS):
    url = f"https://{region}.api.riotgames.com/tft/match/v1/matches/{matchId}"
    r = requests.get(url, headers=HEADERS)
    if r.status_code == 200:
        return r.json()
    else:
        return f"[Error] get_tft_match_by_id: {r.status_code} {r.text}"


In [11]:
listOfMatchData = [get_tft_match_by_id(region, x, HEADERS) for x in listtOfMatches]


In [12]:
listOfMatchData

[{'metadata': {'data_version': '6',
   'match_id': 'PBE1_4508747974',
   'participants': ['V2R9HUNz8xusydyw5opTLQJ6zYN_dwsq86ApT90QbtfpkO-KHKGEDHVKVkTL7H28-VoVggxC-I3b2g',
    'JlunbsYQF0n6PMZnWRFh1U0s8oumVDXUEdetgJpb9wKlQSv65AUA_23dVUwijXgd-I95haLPOIuktg',
    'wcEgXTXwzhA_kkUNkOo_yAow7CTPR9QvtApmqIZoaqP8Wqs0kT6qnC4XqpeyJCN01nzEEQgff4_LDA',
    'FhoOpsMCJoxG_URGQD7QBm6o6L7OOyscaFRSkq3Y4aqSTPJD6C9uxvJNSF0YLuAcu2bRoCgHNqzZlg',
    'dY1iOMlz4B5hASgR0lrkJbswM6aYq7viNgjAIkM6XJhAviNcCLX5xj9HRgImWCWCgLzmfW3fLCIVUA',
    '-JwZwX6sGKr9k2vNFEbSk60Lz4KSJMiVaOnKt85mpt2AmtYoO-pcqJSSZhuQLEvSysgSN0eZIDGK-g',
    '-1orHImEuECQzq4E-Owp5wXuRspdESW9VGXL306bK_f-dU43_G50ezFc6LCrqsj59xwHP3etrs9j-A',
    'bzNYsNQUnSC3sS3sjB2lV0HlAXcIcupZYPg5e5qYWjvNi-r-M6JMG8LbhgR0SjRGbJsdqXc-kU4NIw']},
  'info': {'endOfGameResult': 'GameComplete',
   'gameCreation': 1753758415000,
   'gameId': 4508747974,
   'game_datetime': 1753760701619,
   'game_length': 2246.216552734375,
   'game_version': 'Linux Version 15.15.699.078

In [9]:

# # --- Start Process ---
# seen_puuids = set()
# queue = []

# # Step 1: Get initial summoner
# summoner = get_summoner_by_name(gameName)
# if not summoner:
#     exit("Could not retrieve initial summoner.")
#     print(sum)
# puuid = summoner["puuid"]
# seen_puuids.add(puuid)
# queue.append(puuid)

# # Breadth-first loop through PUUIDs
# while queue:
#     current_puuid = queue.pop(0)
#     match_ids = get_match_ids_by_puuid(current_puuid, count=5)

#     for match_id in match_ids:
#         match_data = get_match_details(match_id)
#         if not match_data:
#             continue
#         participants = match_data.get("metadata", {}).get("participants", [])
        
#         for p_puuid in participants:
#             if p_puuid not in seen_puuids:
#                 seen_puuids.add(p_puuid)
#                 queue.append(p_puuid)

#             account_info = get_account_by_puuid(p_puuid)
#             if not account_info:
#                 continue

#             summoner_info = get_summoner_by_name(account_info["gameName"])
#             if not summoner_info:
#                 continue

#             # Save data to CSV
#             csv_writer.writerow({
#                 "puuid": p_puuid,
#                 "gameName": account_info["gameName"],
#                 "tagLine": account_info["tagLine"],
#                 "summonerLevel": summoner_info["summonerLevel"],
#                 "match_id": match_id
#             })

#         time.sleep(1.2)  # rate limiting

# csv_file.close()
# print("✅ Done. Data saved to 'tft_match_data.csv'")
